# 16. 순환 신경망으로 순차 데이터 모델링

## 16.1 순차 데이터 소개

## 16.2 시퀀스 모델링을 위한 RNN

### 16.2.3 은닉 순환과 출력 순환

In [1]:
import tensorflow as tf
tf.random.set_seed(1)
rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True,
    return_sequences=True)
rnn_layer.build(input_shape=(None, None, 5))
w_xh, w_oo, b_h = rnn_layer.weights
print('W_xh 크기: ', w_xh.shape)
print('W_oo 크기: ', w_oo.shape)
print('b_h 크기: ', b_h.shape)

2022-04-27 21:17:41.541085: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


W_xh 크기:  (5, 2)
W_oo 크기:  (2, 2)
b_h 크기:  (2,)


In [2]:
# 수동으로 구현하기
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5], dtype=tf.float32)
## simple RNN의 출력
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))
## 수동으로 출력 계산하기
out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print('타임 스텝 {} =>'.format(t))
    print('  입력  : ', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h
    print('  은닉  : ', ht.numpy())
    
    if t > 0:
        prev_o = out_man[t-1]
    else:
        prev_o = tf.zeros(shape=(ht.shape))
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print('  출력 (수동)  :', ot.numpy())
    print('  SimpleRNN 출력: '.format(t), output[0][t].numpy())
    print()

타임 스텝 0 =>
  입력  :  [[1. 1. 1. 1. 1.]]
  은닉  :  [[0.41464037 0.96012145]]
  출력 (수동)  : [[0.39240566 0.74433106]]
  SimpleRNN 출력:  [0.39240566 0.74433106]

타임 스텝 1 =>
  입력  :  [[2. 2. 2. 2. 2.]]
  은닉  :  [[0.82928073 1.9202429 ]]
  출력 (수동)  : [[0.80116504 0.9912947 ]]
  SimpleRNN 출력:  [0.80116504 0.9912947 ]

타임 스텝 2 =>
  입력  :  [[3. 3. 3. 3. 3.]]
  은닉  :  [[1.243921  2.8803642]]
  출력 (수동)  : [[0.95468265 0.9993069 ]]
  SimpleRNN 출력:  [0.95468265 0.9993069 ]



## 16.3 텐서플로로 시퀀스 모델링을 위한 RNN 구현

### 16.3.1 첫 번째 프로젝트: IMDb 영화 리뷰의 감성 분석

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [4]:
# 1 단계 - 데이터셋 만들기
target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices(
    (df.values, target.values))

# 확인
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


In [5]:
# 훈련, 검증, 테스트셋 나누기
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(
    50000, reshuffle_each_iteration=False)
ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)